--------
# Transform - Presidential Elections & Voters' Roll Data
---------------

-----
## 1. Presidential data
---------

### a) Import libraries

In [344]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
import os
import pivottablejs as pvt
from tabula import read_pdf

%matplotlib inline

In [338]:
# set the right foldername
DATA = './presidential/'
MAIN = './'

In [27]:
# check to see files
os.listdir(DATA)

['Bulawayo Metropolitan Province 2018 Harmonised Presidential Election Results.xlsx',
 'Harare Metropolitan Province July 2018 Harmonised Presidential Election Results.xlsx',
 'Manicaland Province Province July 2018 Harmonised Presidential Election Results.xlsx',
 'Mashonaland Central Province July 2018 Harmonised Presidential Election Results.xlsx',
 'Mashonaland East Province July 2018 Harmonised Presidential Election Results.xlsx',
 'Mashonaland West  Province July 2018 Harmonised Presidential Election Results.xlsx',
 'Masvingo Province July 2018 Harmonised Presidential Election Results.xlsx',
 'Matabeleland North Province July 2018 Harmonised Presidential Election Results.xlsx',
 'Matabeleland South Province July 2018 Harmonised Presidential Election Results.xlsx',
 'Midlands Province July 2018 Harmonised Presidential Election Results.xlsx']

### b) Function to Convert Excel Files to Dataframes

In [28]:
def excelToDF(xlsfile, cols):
    """ Takes an Excel File from the ZEC Election results for Zimbabwe presidential elections (one file for each province)
        Processes one file and returns two files, one with raw disaggregated data at polling station level
        And another with aggregated data at constituency and other levels that were preexisting in the
        original files
    """
    # import the provincial results file
    prov = pd.ExcelFile(xlsfile)
    #prov.sheet_names - shows that the first sheetname is the one of interest
    df = prov.parse(sheetname=prov.sheet_names[0])
    df = df.iloc[:,:len(cols)].copy()
    
    # change column names to correct ones, locate row with proper column names first
    column_index = np.where(df.iloc[:,0] == 'DISTRICT')[0][0]
    df.columns = cols

    # get propoer columns and remove null columns
    cols = [c for c in df.columns if type(c)==type('c') ]

    # make sure columns names have no spaces of or punctuation
    df = df.loc[:,cols].copy()
    df.columns = [c.lower().strip().replace(" ","_").replace(".","") for c in cols]
    
    # remove first few columns which are useless
    #column_index = np.where(df.iloc[:,0] == 'DISTRICT')[0][0]
    df = df[column_index+2:].copy()
    
    # get name of province
    namelist = xlsfile.split(" ")
    remove = ['July' ,'2018', 'Harmonised', 'Presidential', 'Election', 'Results.xlsx']
    name =" ".join([n for n in namelist if n not in remove])
     
    print("Processing dfs for {} province".format(name))
    
    # find the rows relevant for raw, disaggregated polling station data
    raw = (df.district.notnull() & df.constituency.notnull() & 
                df.local_authority.notnull() & df.ward_no.notnull() & df.polling_stations.notnull())
    
    # Make the raw and aggregated dfs
    draw = df[raw]
   
    # make column for the province and add to the dfs
    draw['province'] = name
      
    return draw

### c) Define a set of standard colums

In [29]:
# Get columns names to apply to all dfs
xlsfile = DATA+'Bulawayo Metropolitan Province 2018 Harmonised Presidential Election Results.xlsx'
# import the provincial results file
prov = pd.ExcelFile(xlsfile)
#prov.sheet_names - shows that the first sheetname is the one of interest
df = prov.parse(sheetname=prov.sheet_names[0])

# change column names to correct ones
column_index = np.where(df.iloc[:,0] == 'DISTRICT')[0][0]
df.columns = df.loc[column_index]

# get propoer columns and remove null columns
cols = [c for c in df.columns if type(c)==type('c') ]
cols

['DISTRICT',
 'CONSTITUENCY ',
 'LOCAL AUTHORITY',
 'WARD NO.',
 'POLLING STATIONS',
 'STATION CODE',
 'Busha Joseph Makamba FreeZim Congress',
 'Chamisa Nelson MDC Alliance',
 'Chikanga  Everisto Washington Rebuild Zimbabwe',
 'Dzapasi Melbah # 1980 Freedom Movement Zimbabwe',
 'Gava Peter Mapfumo UDF',
 'Hlabangana Kwanele RPZ',
 'Kasiyamhuru Blessing ZPP',
 'Khupe Thokozani MDC-T',
 'Madhuku Lovemore NCA',
 'Mangoma Elton Steers Coalition of Democrats',
 'Manyika Noah Ngoni BZA',
 'Mapfumo Chiguvare Tonderayi Johannes Timothy PPPZ',
 'MARIYACHA Violet UDM',
 'Mhambi-Hove Divine NAPDR',
 'Mnangagwa Emmerson Dambudzo ZANU PF',
 'Moyo Donald Nkosana APA',
 'Mteki Bryn Taurai Independent',
 'Mugadza Willard Tawonezvi BCP',
 'Mujuru Joice Teurai Ropa PRC',
 'Munyanduri Tenda Peter NPF',
 'MutinhirI Ambrose NPF',
 'Shumba Kuzozvirava Doniel UDA',
 'Wilson Peter Harry DOP',
 'Total Votes Rejected ',
 'Ballot Paper Unaccounted for',
 'Total Votes Cast',
 'Total Valid Votes Cast']

### d) Convert all Excel Files to DataFrames

In [30]:
# make sure the folder only contains the excel files of interest
# make a list of the filename to loop through later
# grab only the excel files
provincial_filenames = [DATA+f for f in os.listdir(DATA) if 'xls' in f]

In [31]:
provincial_filenames

['./presidential/Bulawayo Metropolitan Province 2018 Harmonised Presidential Election Results.xlsx',
 './presidential/Harare Metropolitan Province July 2018 Harmonised Presidential Election Results.xlsx',
 './presidential/Manicaland Province Province July 2018 Harmonised Presidential Election Results.xlsx',
 './presidential/Mashonaland Central Province July 2018 Harmonised Presidential Election Results.xlsx',
 './presidential/Mashonaland East Province July 2018 Harmonised Presidential Election Results.xlsx',
 './presidential/Mashonaland West  Province July 2018 Harmonised Presidential Election Results.xlsx',
 './presidential/Masvingo Province July 2018 Harmonised Presidential Election Results.xlsx',
 './presidential/Matabeleland North Province July 2018 Harmonised Presidential Election Results.xlsx',
 './presidential/Matabeleland South Province July 2018 Harmonised Presidential Election Results.xlsx',
 './presidential/Midlands Province July 2018 Harmonised Presidential Election Results

In [32]:
# get lists of dfs, each containing an aggregated and disaggregated dataframe
# each list index represents a province
#agg_dfs = [excelToDF(fn, cols)[1] for fn in provincial_filenames]
raw_dfs = [excelToDF(fn, cols) for fn in provincial_filenames]


Processing dfs for ./presidential/Bulawayo Metropolitan Province province
Processing dfs for ./presidential/Harare Metropolitan Province province
Processing dfs for ./presidential/Manicaland Province Province province
Processing dfs for ./presidential/Mashonaland Central Province province
Processing dfs for ./presidential/Mashonaland East Province province
Processing dfs for ./presidential/Mashonaland West  Province province
Processing dfs for ./presidential/Masvingo Province province
Processing dfs for ./presidential/Matabeleland North Province province
Processing dfs for ./presidential/Matabeleland South Province province
Processing dfs for ./presidential/Midlands Province province


### e) Cancatenate DataFrames and Transfer to Excel for Analysis

In [36]:
# Concatenate
rconcat = pd.concat(raw_dfs)
rconcat = rconcat.reset_index().drop(['index'], axis=1)

# Convert and save to Excel in the main folder
rconcat.to_excel(MAIN+"zimelectionresults2018_bypollingstation.xlsx")

----
# 2 - Voters' Roll Data
-----

### a) Read in Voters Roll data 

In [2]:
# set foldername
VOTERSDATA = './votersroll/'

In [6]:
voterfiles = [VOTERSDATA+f for f in os.listdir(VOTERSDATA) if 'Roll.xlsx' in f]

In [7]:
voterfiles

["./votersroll/Bulawayo Metropolitan Province Consolidated Voters' Roll.xlsx",
 "./votersroll/Harare Metropolitan Province Consolidated Voters' Roll.xlsx",
 "./votersroll/Manicaland Province Consolidated Voters' Roll.xlsx",
 "./votersroll/Mashonaland Central Province Consolidated Voters' Roll.xlsx",
 "./votersroll/Mashonaland East Province Consolidated Voters' Roll.xlsx",
 "./votersroll/Mashonaland West Province Consolidated Voters' Roll.xlsx",
 "./votersroll/Masvingo Province Consolidated Voters' Roll.xlsx",
 "./votersroll/Matabeleland North Province Consolidated Voters' Roll.xlsx",
 "./votersroll/Matabeleland South Province Consolidated Voters' Roll.xlsx",
 "./votersroll/Midlands Province Consolidated Voters' Roll.xlsx"]

In [12]:
# This piece of code takes a long time
# So uncomment to run then comment again to avoid running it by mistake
'''
voterdfs = [pd.read_excel(f) for f in voterfiles]
'''

### b) Clean up and consolidate all provinces into one file

In [240]:
len(voterdfs)

10

In [352]:
# Check consistency of file structure.
# The second row of each file must contain the desired columnnames
[",".join(list(d.iloc[1,:])) for d in voterdfs]

In [154]:
# Get the right columnn names, in right order
new_cols = list(voterdfs[1].iloc[1,:]); new_cols

['Ser',
 'Surname',
 'Forenames',
 'ID Number',
 'Gender',
 'Date of Birth',
 'Voter Address',
 'Polling Station Code',
 'Polling Station Name',
 'Ward',
 'Local Authority',
 'Constituency',
 'District',
 'Province ']

In [203]:
# Assign new columns to each df
for d in voterdfs:
    d.columns = new_cols

voterdfs = [d.iloc[2:,:] for d in voterdfs]

In [243]:
voterdfs[0].head(3)

,Ser,Surname,Forenames,ID Number,Gender,Date of Birth,Voter Address,Polling Station Code,Polling Station Name,Ward,Local Authority,Constituency,District,Province
2,1,ALI BABA,SANTI,08-012142 -Z00,Female,1957-10-06 00:00:00,"FLAT NUMBER 8, GEORGE SILUNDIKA, CBD, BULAWAYO",0801BYO0101 A,City Hall,1,Bulawayo Municipality,Bulawayo Central,Bulawayo,Bulawayo Metropolitan
3,2,ASANI,ALLAN,29-2039989-V29,Male,1998-09-12 00:00:00,"NUMBER 503 CIPF, JASON MOYO, CBD, BULAWAYO",0801BYO0101 A,City Hall,1,Bulawayo Municipality,Bulawayo Central,Bulawayo,Bulawayo Metropolitan
4,3,ASHLEY,ARTHUR RAYMOND,75-068008 -K00,Male,1954-10-15 00:00:00,"HOUSE NUMBER 17 A, SECOND AVENUE, CITY CENTR...",0801BYO0101 A,City Hall,1,Bulawayo Municipality,Bulawayo Central,Bulawayo,Bulawayo Metropolitan


In [244]:
# Make the dataframe for votersroll
df_vroll = pd.concat(voterdfs)

In [245]:
df_vroll['Province '].unique()

array(['Bulawayo Metropolitan ', 'Harare Metropolitan ', 'Manicaland ',
       'Mashonaland Central ', 'Mashonaland East ', 'Mashonaland West ',
       'Masvingo ', 'Matabeleland North ', 'Matabeleland South ',
       'Midlands '], dtype=object)

In [246]:
# Remove punctuation from the column names
df_vroll.columns = [c.lower().strip().replace(" ","_").replace(".","") for c in df_vroll.columns]

In [247]:
df_vroll.head(1)

,ser,surname,forenames,id_number,gender,date_of_birth,voter_address,polling_station_code,polling_station_name,ward,local_authority,constituency,district,province
2,1,ALI BABA,SANTI,08-012142 -Z00,Female,1957-10-06 00:00:00,"FLAT NUMBER 8, GEORGE SILUNDIKA, CBD, BULAWAYO",0801BYO0101 A,City Hall,1,Bulawayo Municipality,Bulawayo Central,Bulawayo,Bulawayo Metropolitan


In [249]:
# remove leftover stuff from the province files
df_vroll = df_vroll[(df_vroll["forenames"] !='Forenames') & (df_vroll["surname"] !='Surname') & ((df_vroll["forenames"].notnull()) | (df_vroll["surname"].notnull()))]

### c) Glimpse of the data, check for nulls and duplicates

In [250]:
df_vroll.shape

(5695706, 14)

In [251]:
# Quick data summary
df_vroll.describe()

,ser,surname,forenames,id_number,gender,date_of_birth,voter_address,polling_station_code,polling_station_name,ward,local_authority,constituency,district,province
count,5695706,5695706,5695704,5695706,5695706,5695706,5695699,5695706,5695706,5695706,5695706,5695706,5695706,5695706
unique,900728,211422,761416,5695701,2,29754,2741207,10985,8986,46,89,210,63,10
top,1,MOYO,TENDAI,26-209414 -S26,Female,1970-01-01 00:00:00,"ZIMBIRU VILLAGE, HEADMAN MURAPE, CHIEF CHINAMH...",4700MRE1102,DDF Manyame Training Centre,1,Harare Municipality,Harare South,Harare,Harare Metropolitan
freq,10,115524,32119,2,3073190,1959,1304,1000,9463,291066,684282,76425,756301,900728


In [252]:
# Null entries
df_vroll.isnull().sum()

ser                     0
surname                 0
forenames               2
id_number               0
gender                  0
date_of_birth           0
voter_address           7
polling_station_code    0
polling_station_name    0
ward                    0
local_authority         0
constituency            0
district                0
province                0
dtype: int64

In [253]:
df_vroll[df_vroll["forenames"].isnull()]

,ser,surname,forenames,id_number,gender,date_of_birth,voter_address,polling_station_code,polling_station_name,ward,local_authority,constituency,district,province
283008,283007,SIBANDA,NaN,73-004591 -Q73,Female,1946-01-12 00:00:00,"VILLAGE NJOBO, HEADMAN SIKENTE, CHIEF GAMPO",7300TSH1005,Sikente Primary School,10,Tsholotsho RDC,Tsholotsho South,Tsholotsho,Matabeleland North
231080,231079,NDLOVU,NaN,28-030337 -G39,Female,1958-06-01 00:00:00,"KAPENI VILLAGE, HEADMAN NORMAN NDLOVU, CHIE...",3900MTB0501,Beula Clinic Tent,5,Matobo RDC,Matobo South,Matobo,Matabeleland South


In [258]:
# Discover posible duplication, clean up dirty columns
df_dup =df_vroll[df_vroll["id_number"].duplicated(keep=False)]; df_dup

,ser,surname,forenames,id_number,gender,date_of_birth,voter_address,polling_station_code,polling_station_name,ward,local_authority,constituency,district,province
117621,117620,CHINGANDU,TANYANYIWA MOSES,63-493581 -L25,Male,1940-01-12 00:00:00,"HOUSE NUMBER 3271, NHUNGURU STREET, ZENGEZA, C...",6301CHIT1001 A,Early Learning Centre,10,Chitungwiza Municipality,Zengeza West,Chitungwiza,Harare Metropolitan
118634,118633,CHINGANDU,TANYANYIWA MOSES,63-493581 -L25,Male,1940-01-12 00:00:00,"HOUSE NUMBER 21, NHUNGURU STREET, ZENGEZA 2, C...",6301CHIT1002,Mhuriimwe A High School,10,Chitungwiza Municipality,Zengeza West,Chitungwiza,Harare Metropolitan
262096,262095,MAKAMURE,ROSARIO,38-123629 -B38,Female,1978-10-24 00:00:00,"HOUSE NUMBER C022, UNNAMED STREET, EPWORTH, HA...",6301EPW0401 C,Chizungu Primary School,4,Epworth Local Board,Epworth,Harare,Harare Metropolitan
159321,159320,KONDO,CHALTON,38-123629 -B38,Male,1974-06-28 00:00:00,"VILLAGE SUNGWI 1B, HEADMAN MAZAULA, CHIEF MUJINGA",3800HUR0201,Chehamba Primary School,2,Hurungwe RDC,Hurungwe East,Hurungwe,Mashonaland West
59230,59229,CHIKWAVA,TALENT,26-186952 -C26,Male,1990-11-24 00:00:00,"CHINYANGA VILLAGE, HEADMAN CHIREYA, CHIEF CHIREYA",2600GON0406,Mavimbi Primary School,4,Gokwe North RDC,Gokwe-Chireya,Gokwe North,Midlands
85117,85116,SHOKO,MIRRIAM,26-209414 -S26,Female,1997-08-14 00:00:00,"VILLAGE CHIPARANGANDA, HEADMAN GUMUNYU, CHIEF ...",2600GON2201,Copper Queen Mine Tent,22,Gokwe North RDC,Gokwe-Gumunyu,Gokwe North,Midlands
109818,109817,SHOKO,MIRIAM,26-209414 -S26,Female,1997-08-14 00:00:00,"VILLAGE MUTIURA, HEADMAN MAKORE, CHIEF CHIREYA",2600GON0306,Mutiura Dip Tank Tent,3,Gokwe North RDC,Gokwe-Kabuyuni,Gokwe North,Midlands
670643,670642,CHIKWAVA,TALENT,26-186952 -C26,Male,1990-11-24 00:00:00,"VILLAGE 9, TOKWE 3 FARMING AREA, SHURUGWI",6600TON1607,Zviumwa Secondary School,16,Tongogara RDC,Shurugwi North,Shurugwi,Midlands
715256,715255,NDLOVU,TENDAI,67-076653 -E67,Female,1971-08-30 00:00:00,"HOUSE NUMBER 579, MANDAVA",6701ZTC0102,Zvizvobgo Tower Light Tent,1,Zvishavane Town Council,Zvishavane Ngezi,Zvishavane,Midlands
743380,743379,NDLOVU,TENDAI,67-076653 -E67,Female,1971-08-30 00:00:00,"MUDERERI VILLAGE, HEADMAN MUDERERI, CHIEF MASU...",6700RDC1304,Musuki Business Centre Tent,13,Runde RDC,Zvishavane Runde,Zvishavane,Midlands


In [259]:
# Lsts remove the following
id_dups = [118634,159321, 109818, 670643, 743380]

In [262]:
# Drop the duplicates
df_vroll = df_vroll.drop(id_dups, axis=0); df_vroll.shape

(5695671, 14)

In [264]:
# Check provinces
df_vroll.province.unique()

array(['Bulawayo Metropolitan ', 'Harare Metropolitan ', 'Manicaland ',
       'Mashonaland Central ', 'Mashonaland East ', 'Mashonaland West ',
       'Masvingo ', 'Matabeleland North ', 'Matabeleland South ',
       'Midlands '], dtype=object)

### d) De-identify the data, save and aggreagate at polling station level

In [266]:
# Remove names, and address
df_vroll = df_vroll.drop(["surname", "forenames", "voter_address", "ser"], axis=1)
df_vroll.shape

(5695671, 10)

In [268]:
df_vroll.head(3)

,id_number,gender,date_of_birth,polling_station_code,polling_station_name,ward,local_authority,constituency,district,province
2,08-012142 -Z00,Female,1957-10-06 00:00:00,0801BYO0101 A,City Hall,1,Bulawayo Municipality,Bulawayo Central,Bulawayo,Bulawayo Metropolitan
3,29-2039989-V29,Male,1998-09-12 00:00:00,0801BYO0101 A,City Hall,1,Bulawayo Municipality,Bulawayo Central,Bulawayo,Bulawayo Metropolitan
4,75-068008 -K00,Male,1954-10-15 00:00:00,0801BYO0101 A,City Hall,1,Bulawayo Municipality,Bulawayo Central,Bulawayo,Bulawayo Metropolitan


### e) Save deidentified data per province

In [271]:
# Get province list
provinces =list(df_vroll.province.unique()); provinces

['Bulawayo Metropolitan ',
 'Harare Metropolitan ',
 'Manicaland ',
 'Mashonaland Central ',
 'Mashonaland East ',
 'Mashonaland West ',
 'Masvingo ',
 'Matabeleland North ',
 'Matabeleland South ',
 'Midlands ']

In [275]:
#Uncommenting this since it takes a lot of time to run
# Make sure you don't run by mistake
# Save Excel file for each province
'''
for prov in provinces:
    df = df_vroll[df_vroll['province']==prov]
    df.to_excel(VOTERSDATA+prov+" 2018 Voters' Roll_deidentified.xlsx")
    '''

In [279]:
df_vroll.head()

,id_number,gender,date_of_birth,polling_station_code,polling_station_name,ward,local_authority,constituency,district,province
2,08-012142 -Z00,Female,1957-10-06 00:00:00,0801BYO0101 A,City Hall,1,Bulawayo Municipality,Bulawayo Central,Bulawayo,Bulawayo Metropolitan
3,29-2039989-V29,Male,1998-09-12 00:00:00,0801BYO0101 A,City Hall,1,Bulawayo Municipality,Bulawayo Central,Bulawayo,Bulawayo Metropolitan
4,75-068008 -K00,Male,1954-10-15 00:00:00,0801BYO0101 A,City Hall,1,Bulawayo Municipality,Bulawayo Central,Bulawayo,Bulawayo Metropolitan
5,08-2031613-W00,Female,1985-11-27 00:00:00,0801BYO0101 A,City Hall,1,Bulawayo Municipality,Bulawayo Central,Bulawayo,Bulawayo Metropolitan
6,63-341878 -R63,Female,1965-03-12 00:00:00,0801BYO0101 A,City Hall,1,Bulawayo Municipality,Bulawayo Central,Bulawayo,Bulawayo Metropolitan


### f) Engineer features like Age and Gender

In [284]:
df_vroll['male'], df_vroll['female'] = df_vroll['gender'] =='Male' , df_vroll['gender'] =='Female' 

In [318]:
# Uncomment below, the code takes a while as it loops throough
'''
age = df_vroll['date_of_birth'].apply(lambda x: (pd.to_datetime('2018-07-31 00:00:00')-x).days/365.25)
df_vroll.loc[:,'age'] = age
'''

In [324]:
# Voters by Poliing station
df_vroll_by_ps = df_vroll.groupby(['polling_station_code','polling_station_name',
                  'ward','local_authority','constituency','province']).agg(['count', 'sum', 'mean']).reindex()

In [328]:
df_vroll_by_ps = df_vroll_by_ps.reset_index()

In [330]:
df_vroll_by_ps.head()

polling_station_code          polling_station_name ward local_authority  \
                                                                            
0          0200BBE0101  Chikwalakwala Primary School    1  Beitbridge RDC   
1          0200BBE0102        Chipise Primary School    1  Beitbridge RDC   
2          0200BBE0103    Chituripasi Primary School    1  Beitbridge RDC   
3          0200BBE0104        Lungowe Primary School    1  Beitbridge RDC   
4          0200BBE0105         Malabe Primary School    1  Beitbridge RDC   

      constituency             province  male                  female         \
                                        count    sum      mean  count    sum   
0  Beitbridge East  Matabeleland South    206   68.0  0.330097    206  138.0   
1  Beitbridge East  Matabeleland South    276  123.0  0.445652    276  153.0   
2  Beitbridge East  Matabeleland South    474  185.0  0.390295    474  289.0   
3  Beitbridge East  Matabeleland South    348  112.0  0.321839    348  236.0   
4  Beitbridge East  Matabeleland South    285  107.0  0.375439    285  178.0   

              age                           
       mean count           sum       mean  
0  0.669903   206   9226.579055  44.789219  
1  0.554348   276  12003.069131  43.489381  
2  0.609705   474  21651.394935  45.678048  
3  0.678161   348  15123.638604  43.458732  
4  0.624561   285  12478.012320  43.782499

In [357]:
# Rename the columns
final_cols = ['polling_station_code',  'polling_station_name',  'ward', 'local_authority', 'constituency', ,'province', 'tot_count', 'male_count',
'male_pct','tot_count2', 'female_count', 'female_pct','tot_count3', 'total_age', 'mean_age']
df_vroll_by_ps.columns = final_cols
df_vroll_by_ps.head()

,polling_station_code,polling_station_name,ward,local_authority,constituency,province,tot_count,male_count,male_pct,female_count,female_pct,total_age,mean_age
0,0200BBE0101,Chikwalakwala Primary School,1,Beitbridge RDC,Beitbridge East,Matabeleland South,206,68.0,0.330097,138.0,0.669903,9226.579055,44.789219
1,0200BBE0102,Chipise Primary School,1,Beitbridge RDC,Beitbridge East,Matabeleland South,276,123.0,0.445652,153.0,0.554348,12003.069131,43.489381
2,0200BBE0103,Chituripasi Primary School,1,Beitbridge RDC,Beitbridge East,Matabeleland South,474,185.0,0.390295,289.0,0.609705,21651.394935,45.678048
3,0200BBE0104,Lungowe Primary School,1,Beitbridge RDC,Beitbridge East,Matabeleland South,348,112.0,0.321839,236.0,0.678161,15123.638604,43.458732
4,0200BBE0105,Malabe Primary School,1,Beitbridge RDC,Beitbridge East,Matabeleland South,285,107.0,0.375439,178.0,0.624561,12478.012320,43.782499


In [335]:
# Remove redundant columns
df_vroll_by_ps = df_vroll_by_ps.drop(['tot_count2', 'tot_count3'], axis=1); df_vroll_by_ps.head()

,province,constituency,local_authority,ward,polling_station_name,polling_station_code,tot_count,male_count,male_pct,female_count,female_pct,total_age,mean_age
0,0200BBE0101,Chikwalakwala Primary School,1,Beitbridge RDC,Beitbridge East,Matabeleland South,206,68.0,0.330097,138.0,0.669903,9226.579055,44.789219
1,0200BBE0102,Chipise Primary School,1,Beitbridge RDC,Beitbridge East,Matabeleland South,276,123.0,0.445652,153.0,0.554348,12003.069131,43.489381
2,0200BBE0103,Chituripasi Primary School,1,Beitbridge RDC,Beitbridge East,Matabeleland South,474,185.0,0.390295,289.0,0.609705,21651.394935,45.678048
3,0200BBE0104,Lungowe Primary School,1,Beitbridge RDC,Beitbridge East,Matabeleland South,348,112.0,0.321839,236.0,0.678161,15123.638604,43.458732
4,0200BBE0105,Malabe Primary School,1,Beitbridge RDC,Beitbridge East,Matabeleland South,285,107.0,0.375439,178.0,0.624561,12478.012320,43.782499


In [358]:
# Save to file
df_vroll_by_ps.to_excel(MAIN+"zimvotersroll2018_bypollingstation.xlsx")

----
## 3- Finally, Use HD5 to Save The data we already engineered at individual level
------

In [345]:
df_vroll.shape

(5695671, 13)

In [348]:
# Write to file
df.to_hdf('zimvotersroll2018_byvoter.h5', key='df_vroll', mode='w')

In [349]:
# To read use:
'''
pd.read_hdf('zimvotersroll2018_byvoter.h5', key='df_vroll')
'''

"\npd.read_hdf('zimvotersroll2018_byvoter.h5', key='df_vroll')\n"

In [347]:
df_vroll.head(2)

,id_number,gender,date_of_birth,polling_station_code,polling_station_name,ward,local_authority,constituency,district,province,male,female,age
2,08-012142 -Z00,Female,1957-10-06 00:00:00,0801BYO0101 A,City Hall,1,Bulawayo Municipality,Bulawayo Central,Bulawayo,Bulawayo Metropolitan,False,True,60.815880
3,29-2039989-V29,Male,1998-09-12 00:00:00,0801BYO0101 A,City Hall,1,Bulawayo Municipality,Bulawayo Central,Bulawayo,Bulawayo Metropolitan,True,False,19.882272


In [346]:
df_vroll_by_ps.head(2)

,province,constituency,local_authority,ward,polling_station_name,polling_station_code,tot_count,male_count,male_pct,female_count,female_pct,total_age,mean_age
0,0200BBE0101,Chikwalakwala Primary School,1,Beitbridge RDC,Beitbridge East,Matabeleland South,206,68.0,0.330097,138.0,0.669903,9226.579055,44.789219
1,0200BBE0102,Chipise Primary School,1,Beitbridge RDC,Beitbridge East,Matabeleland South,276,123.0,0.445652,153.0,0.554348,12003.069131,43.489381


---
## END OF TRANSFORM SCRIPT